# **Web Server Assigment**

# 1




## Overview
In this assignment, you will explore the fundamentals of socket programming for TCP connections in Python. You will learn how to create a socket, bind it to a specific address and port, and send and receive HTTP packets. Additionally, you will gain an understanding of the basic structure of HTTP header formats.

Your task is to develop a web server that processes one HTTP request at a time. The web server should:

1. Accept and parse the incoming HTTP request.
2. Retrieve the requested file from the server’s file system.
3. Construct an HTTP response message comprising the requested file preceded by appropriate header lines.
4. Send the response directly to the client.

If the requested file is not found on the server, the web server must return an HTTP "404 Not Found" message to the client.

## Code
Below is a skeleton code for the web server. Your task is to complete the code. The sections where you need to add your implementation are marked with `#-----#`. Some sections may require multiple lines of code.

## Running the Server
1. Place an HTML file (e.g., `HelloWorld.html`) in the same directory as the server program.
2. Execute the server program.
3. Determine the IP address of the machine running the server (e.g., `128.238.251.26`).
4. From a different machine or the same machine, open a web browser and navigate to the URL corresponding to the server. For example:
   ```
   http://128.238.251.26:6789/HelloWorld.html
   ```
   In this example, `HelloWorld.html` is the name of the HTML file you placed in the server directory. Note the port number `6789` used after the colon. Replace this port number with the one configured in your server code.

5. The browser should display the contents of `HelloWorld.html`. If the port number is omitted (e.g., `http://128.238.251.26/HelloWorld.html`), the browser will default to port 80. In this case, ensure your server is listening on port 80.
6. Test requesting a file that is not present on the server. You should receive a "404 Not Found" message in the browser.


## What to Submit
Submit the following items:

1. The complete server code.
2. Screenshots of your client browser demonstrating:
   - Successful retrieval of the HTML file content from the server.
   - Receiving a "404 Not Found" message for a non-existent file.

---

In [ ]:
# import socket module
from socket import *
import sys  # In order to terminate the program

serverPort = 12000
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(True)
# Prepare a server socket
print('Ready to serve...')

while True:
    # Establish the connection
    print ('Ready to serve...')
    connectionSocket, addr = serverSocket.accept()
    try:
        message = connectionSocket.recv(1024).decode()  # Receive the request message from the client
        print(message)
        filename = message.split()[1]  # Get the requested filename
        f = open(filename[1:])  # Open the file (filename[1:] to remove the leading '/')
        outputdata = f.readlines()  # Read the file content

        # Send one HTTP header line into socket
        connectionSocket.send("HTTP/1.1 200 OK\r\n".encode())  # Sending HTTP 200 response
        connectionSocket.send("Content-Type: text/html\r\n".encode())  # Content type header
        connectionSocket.send("\r\n".encode())  # End of headers

        # Send the content of the requested file to the client
        for i in range(0, len(outputdata)):
            connectionSocket.send(outputdata[i].encode())
        connectionSocket.send("\r\n".encode())  # Send an empty line to end the response

        connectionSocket.close()
    except IOError:
        # Send response message for file not found
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n".encode())
        connectionSocket.send("Content-Type: text/html\r\n".encode())
        connectionSocket.send("\r\n".encode())
        connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>".encode())

        # Close client socket
        connectionSocket.close()

serverSocket.close()
sys.exit()  # Terminate the program after sending the corresponding data


![alt text](<Screenshot (382).png>)
![alt text](<Screenshot (383).png>)
![alt text](<Screenshot (385).png>)
![alt text](<Screenshot (386).png>)

# 2

Currently, the web server handles only one HTTP request at a time. Implement a multithreaded server
that is capable of serving multiple requests simultaneously. Using threading, first create a main thread
in which your modified server listens for clients at a fixed port. When it receives a TCP connection
request from a client, it will set up the TCP connection through another port and services the client
request in a separate thread. There will be a separate TCP connection in a separate thread for each
request/response pair.

In [ ]:
from socket import *
import sys
import threading

def handle_client(connectionSocket, addr):
    try:
        message = connectionSocket.recv(1024).decode()  # Receive the request message from the client
        print(f"Request from {addr}:\n{message}")
        filename = message.split()[1]  # Get the requested filename
        f = open(filename[1:])  # Open the file (filename[1:] to remove the leading '/')
        outputdata = f.readlines()  # Read the file content

        # Send HTTP header line
        connectionSocket.send("HTTP/1.1 200 OK\r\n".encode())
        connectionSocket.send("Content-Type: text/html\r\n".encode())
        connectionSocket.send("\r\n".encode())

        # Send the content of the requested file to the client
        for line in outputdata:
            connectionSocket.send(line.encode())
        connectionSocket.send("\r\n".encode())  # End the response

        f.close()
        connectionSocket.close()
    except IOError:
        # Send 404 Not Found response
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n".encode())
        connectionSocket.send("Content-Type: text/html\r\n".encode())
        connectionSocket.send("\r\n".encode())
        connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>".encode())
        connectionSocket.close()

serverPort = 12000
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(5)  # Allow up to 5 simultaneous connections
print("Multithreaded Server is ready to serve...")

while True:
        # Accept incoming connection
        connectionSocket, addr = serverSocket.accept()
        print(f"Connection established with {addr}")

        # Start a new thread for handling the client
        client_thread = threading.Thread(target=handle_client, args=(connectionSocket, addr))
        client_thread.start()

serverSocket.close()
sys.exit()  # Terminate the program


# 3

The Multi Thread Web Server is now set and working but cannot handle changes in the source files. How can we look for changes in the source directory and access it in your code ? Explore Libraries in python

In [ ]:
# watchdog library can be used which monitors changes in a directory (like adding, modifying, or deleting files) and triggers an event whenever a change occurs.

After you've accessed it in your code, Post a notification on the hosted http page to reload if any source directory changes

In [ ]:
# Below is the modified multithreaded server with file change detection and notification support:

Your dynamic http server is now ready

In [ ]:
from socket import *
import sys
import threading
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Global flag to indicate file changes
file_change_detected = False

# File change handler
class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        global file_change_detected
        if not event.is_directory:
            file_change_detected = True
            print(f"File changed: {event.src_path}")

    def on_created(self, event):
        global file_change_detected
        if not event.is_directory:
            file_change_detected = True
            print(f"New file added: {event.src_path}")

    def on_deleted(self, event):
        global file_change_detected
        if not event.is_directory:
            file_change_detected = True
            print(f"File deleted: {event.src_path}")

# Function to handle each client request
def handle_client(connectionSocket, addr):
    global file_change_detected
    try:
        message = connectionSocket.recv(1024).decode()  # Receive the request message
        print(f"Request from {addr}:\n{message}")
        filename = message.split()[1]  # Get the requested filename

        # Notify the user if file changes are detected
        if file_change_detected:
            response = ("HTTP/1.1 200 OK\r\n"
                        "Content-Type: text/html\r\n\r\n"
                        "<html><body>"
                        "<h1>File changes detected! Please reload the page.</h1>"
                        "</body></html>")
            connectionSocket.send(response.encode())
            file_change_detected = False
        else:
            # Serve the requested file
            with open(filename[1:], 'r') as f:
                outputdata = f.readlines()

            # Send HTTP header
            connectionSocket.send("HTTP/1.1 200 OK\r\n".encode())
            connectionSocket.send("Content-Type: text/html\r\n\r\n".encode())

            # Send file content
            for line in outputdata:
                connectionSocket.send(line.encode())
            connectionSocket.send("\r\n".encode())

        connectionSocket.close()
    except IOError:
        # Send 404 Not Found response
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n".encode())
        connectionSocket.send("Content-Type: text/html\r\n\r\n".encode())
        connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>".encode())
        connectionSocket.close()

# Function to start the file watcher
def start_watcher(directory):
    event_handler = FileChangeHandler()
    observer = Observer()
    observer.schedule(event_handler, path=directory, recursive=False)
    observer.start()
    print("File watcher started.")
    return observer

# Main thread to listen for incoming connections
def start_server():
    serverPort = 12000
    serverSocket = socket(AF_INET, SOCK_STREAM)
    serverSocket.bind(("", serverPort))
    serverSocket.listen(5)
    print("Dynamic HTTP Server is ready to serve...")

    # Start the file watcher
    observer = start_watcher(".")  # Watch the current directory

    try:
        while True:
            connectionSocket, addr = serverSocket.accept()
            print(f"Connection established with {addr}")

            # Start a new thread to handle the client
            client_thread = threading.Thread(target=handle_client, args=(connectionSocket, addr))
            client_thread.start()
    except KeyboardInterrupt:
        print("Shutting down the server...")
        observer.stop()  # Stop the file watcher
        observer.join()
        serverSocket.close()
        sys.exit()

if __name__ == "__main__":
    start_server()


![alt text](<Screenshot (393).png>)
![alt text](<Screenshot (394).png>)
![alt text](<Screenshot (395).png>)
![alt text](<Screenshot (396).png>)